# Introduction

## FASTA file format

In [1]:
from Bio import SeqIO

fname = "../data/01_Introduction/glycoside_hydrolases_nt.fasta"

for record in SeqIO.parse(fname, "fasta"):
    length = len(record.seq)
    print(f"{record.id} length: {length}")
    
print("Done.")

ECA0662 length: 1389
ECA1451 length: 1425
ECA1871 length: 1395
ECA2166 length: 1431
ECA3646 length: 1437
ECA4387 length: 1473
ECA4407 length: 1398
ECA4432 length: 1443
Done.


## GenBank

In [3]:
from Bio import Entrez

Entrez.email = "mattfeng@mit.edu"

In [5]:
accession = "NC_004547.2"
print(f"Fetching {accession} from NCBI...")

fetch_handle = Entrez.efetch("nuccore", id=accession, rettype="gbwithparts", retmode="text")

with open(accession + ".gbk", "w") as output_handle:
    output_handle.write(fetch_handle.read())
    
print(f"Saved {accession}.gbk.")

Fetching NC_004547.2 from NCBI...
Saved NC_004547.2.gbk.


### Parsing GenBank files

In [2]:
from Bio import SeqIO

def get_cds_feature_with_qualifier_value(seq_record, name, value):
    # loop over the features
    for feature in genome_record.features:
        if feature.type == "CDS" and value in feature.qualifiers.get(name, []):
            return feature
    return None

In [3]:
genome_record = SeqIO.read("NC_004547.2.gbk", "genbank")
cds_feature = get_cds_feature_with_qualifier_value(genome_record, "old_locus_tag", "ECA0662")
print(cds_feature)

type: CDS
location: [736846:738235](-)
qualifiers:
    Key: EC_number, Value: ['3.2.1.86']
    Key: codon_start, Value: ['1']
    Key: gene, Value: ['ascB']
    Key: inference, Value: ['COORDINATES: similar to AA sequence:RefSeq:WP_000643228.1']
    Key: locus_tag, Value: ['ECA_RS03295']
    Key: note, Value: ['Derived by automated computational analysis using gene prediction method: Protein Homology.']
    Key: old_locus_tag, Value: ['ECA0662']
    Key: product, Value: ['6-phospho-beta-glucosidase']
    Key: protein_id, Value: ['WP_039289952.1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MKAFPDGFLWGGSVAANQVEGAWNEDGKGVSTSDLQLKGVHGPVTERDESISCIKDRAIDFYHQYPQDIQLFAEMGFKVLRTSIAWTRIYPEGNEAEPCEAGLAFYDHLFDEMAKHHIQPLITLSHYEMPYGLVKKLGGWGNRAVIDHFEKYARTVFSRYKDKVKHWLTFNEINMALHSPFTGIGLSGEPSKQDIYQAIHHQLVASARVVKACHDIIPDAKIGNMLLGAVRYPMTCHPKDVLEAQNKNREWLFFGDVQVRGTYPAWIQRYFRENDVELTITAQDKDDLSHTVDFVSFSYYMSGCATFEPEKYQSSRGNIIRLIPNPHLEASEWGWQIDPEGLRFLLNELYDRYQKPLFIVENGLGARDVVEEDGSIHDS

In [4]:
print(cds_feature.location)

[736846:738235](-)


In [5]:
gene_sequence = cds_feature.extract(genome_record.seq)
print("CDS nucleotide sequence:")
print(gene_sequence)
print(f"Start codon is {gene_sequence[:3]}")
print(f"Stop codon is {gene_sequence[-3:]}")

CDS nucleotide sequence:
ATGAAAGCATTCCCCGACGGATTTTTATGGGGCGGTTCAGTCGCAGCAAATCAGGTTGAAGGGGCATGGAATGAAGACGGCAAAGGCGTGTCGACCTCCGATCTTCAGCTAAAGGGCGTGCATGGTCCGGTGACAGAACGCGATGAGAGCATTAGCTGCATCAAAGATCGGGCAATCGATTTTTATCATCAATATCCGCAGGATATACAGCTATTCGCCGAAATGGGCTTCAAGGTGTTACGCACCTCCATCGCCTGGACGCGTATTTATCCCGAAGGCAATGAAGCAGAACCCTGCGAAGCGGGTCTGGCCTTTTACGATCATCTGTTTGATGAAATGGCAAAGCATCATATTCAGCCGCTGATTACGCTGTCGCACTATGAAATGCCGTACGGTCTGGTGAAAAAGTTGGGAGGCTGGGGCAACCGCGCCGTCATCGACCATTTTGAGAAATATGCCCGTACCGTCTTTAGCCGCTACAAAGACAAGGTGAAGCACTGGCTGACCTTCAATGAAATCAACATGGCGCTGCACTCGCCTTTTACGGGTATCGGGCTAAGCGGCGAACCCTCAAAACAGGATATTTATCAGGCCATCCACCATCAGTTGGTTGCCAGTGCACGCGTGGTGAAAGCCTGTCACGACATCATCCCTGATGCCAAAATCGGCAATATGCTGCTGGGCGCGGTGCGCTACCCCATGACCTGTCATCCGAAAGACGTACTGGAAGCGCAGAACAAGAATCGGGAATGGCTGTTCTTCGGTGACGTGCAAGTTCGCGGCACCTATCCGGCGTGGATTCAGCGTTATTTCCGGGAAAATGATGTCGAACTCACGATTACCGCGCAGGACAAAGACGATCTGAGCCACACCGTAGACTTTGTTTCATTCAGCTATTACATGAGTGGCTGTGCGACCTTCGAACCAGAAAAATACCAGTCTTCACGCGGCAACATCATCCGCCTGATTCCTAAC

In [6]:
protein_sequence = gene_sequence.translate(table=11, cds=True)
print("Translated into amino acids:")
print(protein_sequence)

Translated into amino acids:
MKAFPDGFLWGGSVAANQVEGAWNEDGKGVSTSDLQLKGVHGPVTERDESISCIKDRAIDFYHQYPQDIQLFAEMGFKVLRTSIAWTRIYPEGNEAEPCEAGLAFYDHLFDEMAKHHIQPLITLSHYEMPYGLVKKLGGWGNRAVIDHFEKYARTVFSRYKDKVKHWLTFNEINMALHSPFTGIGLSGEPSKQDIYQAIHHQLVASARVVKACHDIIPDAKIGNMLLGAVRYPMTCHPKDVLEAQNKNREWLFFGDVQVRGTYPAWIQRYFRENDVELTITAQDKDDLSHTVDFVSFSYYMSGCATFEPEKYQSSRGNIIRLIPNPHLEASEWGWQIDPEGLRFLLNELYDRYQKPLFIVENGLGARDVVEEDGSIHDSYRIDYLRRHLIQVREAIDDGVELLGYTSWGPIDLVSAGTAQMSKRYGFIHVDRDDEGKGTLERRRKDSFYWYQDVISSNGKSL


In [12]:
old_tags = ["ECA0662", "ECA1451", "ECA1871", "ECA2166",
            "ECA3646", "ECA4387", "ECA4407", "ECA4432"]

with open("nucleotides.fasta", "w") as nto, open("proteins.fasta", "w") as pto:
    for tag in old_tags:
        print(f"Looking at {tag}")
        cds_feature = get_cds_feature_with_qualifier_value(genome_record, "old_locus_tag", tag)
        gene_seq = cds_feature.extract(genome_record.seq)
        protein_seq = gene_seq.translate(table=11, cds=True)

        assert protein_seq == cds_feature.qualifiers["translation"][0]

        nto.write(f">{tag}\n{gene_seq}\n")
        pto.write(f">{tag}\n{protein_seq}\n")

print("Done.")

Looking at ECA0662
Looking at ECA1451
Looking at ECA1871
Looking at ECA2166
Looking at ECA3646
Looking at ECA4387
Looking at ECA4407
Looking at ECA4432
Done.
